<a href="https://colab.research.google.com/github/PARTHIBAN-007/Transformers/blob/main/Translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers[sentencepiece] evaluate datasets accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [1]:
from datasets import load_dataset

raw_dataset = load_dataset("kde4",lang1="en",lang2="fr")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


The repository for kde4 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/kde4.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/210173 [00:00<?, ? examples/s]

In [2]:
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 210173
    })
})

In [3]:
split_dataset = raw_dataset["train"].train_test_split(train_size=0.8,seed = 20)
split_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 168138
    })
    test: Dataset({
        features: ['id', 'translation'],
        num_rows: 42035
    })
})

In [4]:
split_dataset["validation"] = split_dataset.pop("test")

In [5]:
split_dataset["train"][1]["translation"]

{'en': 'The project page could be found at: https: / /www. gna. org/ projects/ kvpnc;.',
 'fr': 'Téléchargez le depuis la section Fichiers (http: / /download. gna. org/ kvpnc/).'}

In [6]:
from transformers import pipeline

model_checkpoint = "Helsinki-NLP/opus-mt-en-fr"
translator = pipeline("translation",model = model_checkpoint)
translator("The project page could be found at: https: / /www. gna. org/ projects/ kvpnc;.")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/301M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/301M [00:00<?, ?B/s]

...

source.spm:   0%|          | 0.00/778k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.34M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Device set to use cuda:0


[{'translation_text': "La page du projet peut être consultée à l'adresse suivante: https: / /www. gna. org/ projects/ kvpnc;."}]

In [7]:
split_dataset["train"][172]["translation"]

{'en': 'This option produces a sort of vibration in the lightness of the star.',
 'fr': "Cette option produit une sorte de vibration dans la lueur de l' étoile"}

In [8]:
translator("This option produces a sort of vibration in the lightness of the star.")

[{'translation_text': "Cette option produit une sorte de vibration dans la légèreté de l'étoile."}]

In [9]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint,return_tensors="pt")

In [10]:
en_sentence = split_dataset["train"][1]["translation"]["en"]
fr_sentence = split_dataset["train"][1]["translation"]["fr"]

inputs = tokenizer(en_sentence,text_target = fr_sentence)
inputs

{'input_ids': [35, 723, 647, 373, 45, 928, 71, 37, 4012, 9, 37, 583, 583, 3390, 3, 49, 19015, 3, 57, 309, 74, 1013, 74, 2635, 973, 529, 364, 222, 50, 3, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [25966, 19, 540, 8, 669, 33355, 24, 11106, 37, 583, 583, 9507, 10571, 3, 49, 19015, 3, 49, 1937, 74, 2635, 973, 529, 13518, 74, 102, 0]}

In [11]:
max_length = 128

def preprocess_function(examples):
  inputs = [ex["en"] for ex in examples["translation"]]
  targets = [ex["fr"] for ex in examples["translation"]]
  model_inputs = tokenizer(
      inputs,text_target = targets,max_length = max_length ,truncation =True
  )
  return model_inputs

In [12]:
tokenized_dataset = split_dataset.map(
    preprocess_function ,
    batched  = True ,
    remove_columns = split_dataset["train"].column_names
)

Map:   0%|          | 0/168138 [00:00<?, ? examples/s]

Map:   0%|          | 0/42035 [00:00<?, ? examples/s]

In [13]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [14]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer =tokenizer ,model = model)

In [15]:
batch = data_collator([tokenized_dataset["train"][i] for i in range(1,3)])
batch.keys()

dict_keys(['input_ids', 'attention_mask', 'labels', 'decoder_input_ids'])

In [16]:
batch["labels"]

tensor([[25966,    19,   540,     8,   669, 33355,    24, 11106,    37,   583,
           583,  9507, 10571,     3,    49, 19015,     3,    49,  1937,    74,
          2635,   973,   529, 13518,    74,   102,     0],
        [14743,   301,   548,     0,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100]])

In [17]:
batch["decoder_input_ids"]

tensor([[59513, 25966,    19,   540,     8,   669, 33355,    24, 11106,    37,
           583,   583,  9507, 10571,     3,    49, 19015,     3,    49,  1937,
            74,  2635,   973,   529, 13518,    74,   102],
        [59513, 14743,   301,   548,     0, 59513, 59513, 59513, 59513, 59513,
         59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513,
         59513, 59513, 59513, 59513, 59513, 59513, 59513]])

In [18]:
for i in range(1,3):
  print(tokenized_dataset["train"][i]["labels"])

[25966, 19, 540, 8, 669, 33355, 24, 11106, 37, 583, 583, 9507, 10571, 3, 49, 19015, 3, 49, 1937, 74, 2635, 973, 529, 13518, 74, 102, 0]
[14743, 301, 548, 0]


In [19]:
!pip install sacrebleu evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 5.6 MB/s eta 0:00:00


In [20]:
import evaluate

metric = evaluate.load("sacrebleu")

In [21]:
predictions = [
    "This plugin lets you translate web pages between several languages automatically."
]
references = [
    [
        "This plugin allows you to automatically translate web pages between several languages."
    ]
]

metric.compute(predictions =predictions , references = references)

{'score': 46.750469682990165,
 'counts': [11, 6, 4, 3],
 'totals': [12, 11, 10, 9],
 'precisions': [91.66666666666667,
  54.54545454545455,
  40.0,
  33.333333333333336],
 'bp': 0.9200444146293233,
 'sys_len': 12,
 'ref_len': 13}

In [22]:
predictions = ["This Plugin"]
references = [
    [
        "This plugin allows you to automatically translate web pages between several languages."
    ]
]

metric.compute(predictions = predictions, references =references)

{'score': 0.0,
 'counts': [1, 0, 0, 0],
 'totals': [2, 1, 0, 0],
 'precisions': [50.0, 50.0, 0.0, 0.0],
 'bp': 0.004086771438464067,
 'sys_len': 2,
 'ref_len': 13}

In [27]:
import numpy as np

def compute_metrics(eval_preds):
  preds , labels = eval_preds

  if isinstance(preds,tuple):
    preds = preds[0]

  decoded_preds = tokenizer.batch_decode(preds,skip_special_tokens = True)

  labels = np.where(labels !=-100 ,labels ,tokenizer.pad_token_id)
  decoded_labels = tokenizer.batch_decode(labels ,skip_special_tokens = True)

  decoded_preds = [pred.strip() for pred in decoded_preds]
  decoded_labels = [[label.strip()] for label in decoded_labels]

  result = metric.compute(predictions = decoded_preds , references = decoded_labels)
  return {"bleu" : result["score"]}


In [28]:
from transformers import Seq2SeqTrainingArguments , Seq2SeqTrainer

args = Seq2SeqTrainingArguments(
    f"marian-finetuned-kde4-en-to-fr",
    evaluation_strategy="no",
    save_strategy = "epoch",
    learning_rate = 1e-5,
    per_device_train_batch_size = 32,
    per_device_eval_batch_size = 64,
    weight_decay = 0.01,
    save_total_limit = 3 ,
    num_train_epochs = 3,
    predict_with_generate = True ,
    fp16 = True ,
    push_to_hub = True
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [29]:
trainer = Seq2SeqTrainer(
    model,
    args ,
    train_dataset = tokenized_dataset["train"],
    eval_dataset = tokenized_dataset["validation"],
    data_collator = data_collator ,
    tokenizer = tokenizer ,
    compute_metrics = compute_metrics
)

<ipython-input-29-38121613d9c3>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
trainer.evaluate(max_length = max_length)

In [ ]:
trainer.push_to_hub(args = "translation",commit_message = "Training Complete")

In [ ]:
from torch.utils.data import DataLoader

tokenized_dataset.set_format("torch")

train_dataloader = DataLoader(
    tokenized_dataset["train"],
    shuffle = True ,
    collate_fn =data_collator,
    batch_size = 8
)

eval_dataloader = DataLoader(
    tokenized_dataset["validation"],
    collate_fn = data_collator ,
    batch_size = 8
)

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)


In [ ]:
from transformers import AdamW

optimizer = AdamW(model.parameters(),lr = 1e-5)

In [ ]:
from accelerate import Accelerator

accelerator = Accelerator()

model.optimizer , train_dataloader , eval_dataloader = accelerator.prepare(
    model ,optimizer , train_dataloader , eval_dataloader
)


In [ ]:
from transformers import get_scheduler

num_train_epochs = 3
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear" ,
    optimizer = optimizer ,
    num_warmup_steps = 0,
    num_training_steps = num_training_steps
)

In [ ]:
from huggingface_hub import Repository , get_full_repo_name

model_name = ""


In [ ]:
output_dir = ""

In [ ]:
def postprocess(predictions,labels):
  predictions = predictions.cpu().numpy()
  labels = labels.cpu().numpy()

  decoded_preds=  tokenizer.batch_decode(predictions, skip_special_tokens = True)

  labels = np.where(labels !=-100,labels ,tokenizer.pad_token_id)
  decoded_labels = tokenizer.batch_decode(labels,skip_special_tokens = True)

  decoded_preds = [pred.strip() for pred in decoded_preds]
  decoded_labels = [[label.strip()] for label in decoded_labels]

  return decoded_preds ,decoded_labels


In [ ]:
from tqdm.auto import tqdm
import torch

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
  model.train()

  for batch in train_loader:
    outputs = model(**batch)
    loss = output.loss()
    accelerator.backward(loss)

    optimizer.step()
    lr_Scheduler.step()
    optimizer.zero_grad()
    progress_bar.update(1)

  model.eval()
  for batch in tqdm(eval_dataloader):
    with torch.no_grad():
      generated_tokens = accelerator.unwarap_model(model).generate(
          batch["input_ids"],
          attention_mask = batch["attention_mask"],
          max_length = 128,
      )
      labels = batch["labels"]

      generated_tokens = accelerator.pad_across_process(
          generrated_tokens,dim= 1,pa_index = tokenizer.pad_token_id
      )

      labels = accelerator.pad_across_processes(labels, dim=1, pad_index=-100)

      predictions_gathered = accelerator.gather(generated_tokens)
      labels_gathered = accelerator.gather(labels)

      decoded_preds, decoded_labels = postprocess(predictions_gathered, labels_gathered)
      metric.add_batch(predictions=decoded_preds, references=decoded_labels)

  results = metric.compute()
  print(f"epoch {epoch}, BLEU score: {results['score']:.2f}")

  accelerator.wait_for_everyone()

  unwrapped_model = accelerator.unwrap_model(model)
  unwrapped_model.save_pretrained(output_dir,save_function =accelerator.save)

  if accelerator.is_main_process:
        tokenizer.save_pretrained(output_dir)
        model.push_to_hub("Parthiban007/distilbert-base-uncased-FineTuned-imdb")
        tokenizer.push_to_hub("Parthiban007/distilbert-base-uncased-FineTuned-imdb")

In [ ]:
from transformers import pipeline

model_checkpoint = ""
translator = pipeline("translation" ,model=checkpoint)
translator("Hugging Face on Natural Language Processing")